# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from sympy import randprime, prime

In [2]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful assistant for an online tutoring platform. "
system_message += "You specialise in Maths and Science concepts for students in grades first to tenth. "
system_message += "Give courteous answers to the academic questions, with a hint of playfulness, no more than 2 sentences. "
system_message += "Don't entertain non academic questions and respond in a subtle tone to check online resources. "
system_message += "If possible, provide the references to online resources. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [4]:
# function to return random prime number between 1 to upper_range
def rand_prime(upper_range):
    return randprime(1,upper_range)

In [5]:
# function to return nth prime
def nth_prime(n):
    return prime(n)

In [6]:
# Dictionary structure for defining the functions to be used as tool

rand_prime_function = {
    "name": "rand_prime",
    "description": "Returns a random prime number between 1 and an upper range provided by user. If user asks for a prime number greater than 10000 or less than 1, ask to check back in later.",
    "parameters": {
        "type": "object",
        "properties": {
            "upper_range": {
                "type": "integer",
                "description": "The upper range for which the user is interested in finding a random prime number",
            },
        },
        "required": ["upper_range"],
        "additionalProperties": False
    }
}

nth_prime_function = {
    "name": "nth_prime",
    "description": "Returns nth prime number with 2 being used as the first prime number. If user asks for a 101th or further prime number, ask to check back in later.",
    "parameters": {
        "type": "object",
        "properties": {
            "n": {
                "type": "integer",
                "description": "The nth prime number user is interested in",
            },
        },
        "required": ["n"],
        "additionalProperties": False
    }
}

In [13]:
# And this is included in a list of tools:

tools = [
    {"type": "function", "function": rand_prime_function},
    {"type": "function", "function": nth_prime_function}
]

In [37]:
# We have to write that function handle_tool_call:

def handle_tool_call_rand_prime(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    upper_range = arguments.get("upper_range")
    result = rand_prime(upper_range)
    response = {
        "role": "tool",
        "content": json.dumps({"result": result}),
        "tool_call_id": tool_call.id
    }
    return response

In [38]:
# We have to write that function handle_tool_call:

def handle_tool_call_nth_prime(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    n = arguments.get("n")
    result = nth_prime(n)
    response = {
        "role": "tool",
        "content": json.dumps({"result": result}),
        "tool_call_id": tool_call.id
    }
    return response

In [39]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        tool_call = message.tool_calls[0]
        function_name = tool_call.function.name
        result = {}
        if function_name == "rand_prime":
            result = handle_tool_call_rand_prime(message)
        elif function_name == "nth_prime":
            result = handle_tool_call_nth_prime(message)
        messages.append(message)
        messages.append(result)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7893

To create a public link, set `share=True` in `launch()`.
